# Laboratório 01: Escalonamento

## Universidade Federal do Amazonas (UFAM)
### Instituto de Computação (IComp)

Disciplina: Sistemas Operacionais

Professor: André Carvalho

Alunos: Airton Filho e Victor Hugo

---

### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

```python

import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria agora 
        self.chegada = 0

    def roda(self,quantum=None): # se rodar sem quantum, o processor roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S
    

class escalonador(object): # Protótipo de escalonador de exemplo
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        print
        # implemente aqui o que escalonador faz quando surge um novo processo pronto

        
    def proximo(self):
        # implemente aqui a politica de escalonamento para definir um processo a ser executado
        return p #processo p eh escolhido
```

---

Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

```python
nprocs = 4
nomes = ['A','B','C','D']
chanceio = [0,0,0,0]  # Valor de zero a cem, chance de ser entrada/saída — por enquanto deixem em zero
tamanho = np.array([3,3,3,3])

total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i], chanceio[i], tamanho[i], 0, 0))  # Cria uma lista procs de Processos
```

Na célula abaixo, temos uma simulação do funcionamento de um escalonador de processos. As duas configurações importantes aqui são o valor do quantum padrão (que pode ser dinamico em algoritmos mais complexos, e quantos timeslices um processo que faz e/s passa bloqueado.

Percebam que na terceira linha é instanciado o escalonador (neste caso, um round_robin). Isto foi feito assim para ser simples trocar o escalonador e repetir a simulação, bastando criar uma classe com os métodos pronto, proximo e construtor e alterar esta linha.

---

```python
quantum = 2
tempoBloq = 1

escalonador = escalonador(procs) #troque escalonador pelo seu escalonador
bloqueados = []

tempo = 0

random.seed(0)

while total>0:
    p = escalonador.proximo()
    if(p is not None):
        rodou, _ = p.roda() #adicione quantum como parâmetro, por enquanto nao temos E/S
        
        if(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        tempo+=rodou
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
```

---

# Laboratório:


1 - Altere o simulador acima para calcular o tempo de execucao medio e tempo de resposta 

2 - Implemente o escalonador por FIFO e SJF e verifique seus tempos de execução e espera.

3 - Faça em outra célula uma implementação do STCF e Round Robin, alterando o p.roda() para receber o quantum


Na segunda feira (15) haverá um questionário no ColabWeb onde você deverá utilizar suas implementações para responder às perguntas. Na terça de manhã farei uma breve conferência para tirar dúvidas.

In [ ]:
import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria agora 
        self.chegada = 0


    # def roda modificado para receber com quantum. por padrão, se não tiver nenhum quanto ele o define como None
    # tal implementação também atende aos escalonadores fifo e SJF, q n usam quantum
    def roda(self,quantum=None): # se rodar sem quantum, o processo roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S

In [26]:
class EscalonadorFIFO:
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()  # cópia da lista original

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if len(self.prontos) > 0:
            return self.prontos.pop(0)
        return None

In [27]:
class EscalonadorSJF:
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()

    def pronto(self, processo):
        self.prontos.append(processo)
        self.prontos.sort(key=lambda p: p.tam)  # menor primeiro

    def proximo(self):
        if len(self.prontos) > 0:
            return self.prontos.pop(0)
        return None


In [28]:
class EscalonadorSTCF:
    def __init__(self, vprontos=[]):
        self.prontos = vprontos.copy()

    def pronto(self, processo):
        self.prontos.append(processo)
        self.prontos.sort(key=lambda p: p.tam)  # ordena a cada novo processo

    def proximo(self):
        if self.prontos:
            return self.prontos.pop(0)
        return None


In [29]:
# round-robin

class EscalonadorRR:
    def __init__(self, vprontos=None, quantum=2):  # Agora recebendo o parâmetro quantum
        if vprontos is None:
            vprontos = []
        self.prontos = vprontos.copy()
        self.quantum = quantum  # Inicializa a variável quantum

    def pronto(self, processo):
        self.prontos.append(processo)

    def proximo(self):
        if self.prontos:
            return self.prontos.pop(0)
        return None


In [30]:
def simular(escalonador, processos, quantum=2, usa_quantum=True):
    tempo = 0
    total = sum(p.tam for p in processos)
    
    tempo_inicio = {}  # resposta
    tempo_final = {}   # turnaround
    tempo_resposta = {}
    
    fila_espera = {p.nome: 0 for p in processos}

    random.seed(0)

    while total > 0:
        p = escalonador.proximo()

        if p is not None and p.tam > 0:
            # tempo de resposta
            if p.nome not in tempo_inicio:
                tempo_inicio[p.nome] = tempo
                tempo_resposta[p.nome] = tempo - p.chegada

            # roda com ou sem quantum
            q = quantum if usa_quantum else None
            rodou, _ = p.roda(q)

            total -= rodou
            tempo += rodou

            # atualiza tempo de espera dos outros
            for outro in processos:
                if outro.nome != p.nome and outro.tam > 0:
                    fila_espera[outro.nome] += rodou

            # reinserir se não terminou
            if p.tam > 0:
                escalonador.pronto(p)
            else:
                tempo_final[p.nome] = tempo
        else:
            tempo += 1

    # cálculo das métricas
    tempos_execucao = {p.nome: tempo_final[p.nome] - p.chegada for p in processos}
    tempos_espera = fila_espera

    print("\n--- Métricas por Processo ---")
    for p in processos:
        print(f"{p.nome} -> Resposta: {tempo_resposta[p.nome]}, Espera: {tempos_espera[p.nome]}, Execução: {tempos_execucao[p.nome]}")

    media_resposta = sum(tempo_resposta.values()) / len(processos)
    media_espera = sum(tempos_espera.values()) / len(processos)
    media_execucao = sum(tempos_execucao.values()) / len(processos)

    print(f"\nTempo médio de resposta: {media_resposta:.2f}")
    print(f"Tempo médio de espera:   {media_espera:.2f}")
    print(f"Tempo médio de execução: {media_execucao:.2f}")

    return tempo_resposta, tempos_espera, tempos_execucao


---

# Questionário 01

Dado um workload de 5 processos de tamanho 20, calcule o tempo de execução do escalonador FIFO:

Dado um workload de 5 processos de tamanho 20, calcule o tempo de resposta do escalonador FIFO:

In [23]:
# Setup
nomes = ['A', 'B', 'C', 'D', 'E']
tamanho = [20] * 5
chanceio = [0] * 5
procs = [Processo(nomes[i], chanceio[i], tamanho[i], 0, 0) for i in range(5)]

# Simulação FIFO
fifo = EscalonadorFIFO(procs.copy())
simular(fifo, procs.copy(), usa_quantum=False)

A  rodou por  20  timeslice, faltam  0
B  rodou por  20  timeslice, faltam  0
C  rodou por  20  timeslice, faltam  0
D  rodou por  20  timeslice, faltam  0
E  rodou por  20  timeslice, faltam  0

--- Métricas por Processo ---
A -> Resposta: 0, Espera: 0, Execução: 20
B -> Resposta: 20, Espera: 20, Execução: 40
C -> Resposta: 40, Espera: 40, Execução: 60
D -> Resposta: 60, Espera: 60, Execução: 80
E -> Resposta: 80, Espera: 80, Execução: 100

Tempo médio de resposta: 40.00
Tempo médio de espera:   40.00
Tempo médio de execução: 60.00


({'A': 0, 'B': 20, 'C': 40, 'D': 60, 'E': 80},
 {'A': 0, 'B': 20, 'C': 40, 'D': 60, 'E': 80},
 {'A': 20, 'B': 40, 'C': 60, 'D': 80, 'E': 100})

---

Dado um workload de 5 processos de tamanho 20, calcule o tempo de execução do escalonador SJF:

Dado um workload de 5 processos de tamanho 20, calcule o tempo de resposta do escalonador SJF:

In [15]:
# Setup
nomes = ['A', 'B', 'C', 'D', 'E']
tamanho = [20] * 5
chanceio = [0] * 5
procs = [Processo(nomes[i], chanceio[i], tamanho[i], 0, 0) for i in range(5)]

sjf = EscalonadorSJF(procs.copy())
simular(sjf, procs.copy(), usa_quantum=False)


A  rodou por  20  timeslice, faltam  0
B  rodou por  20  timeslice, faltam  0
C  rodou por  20  timeslice, faltam  0
D  rodou por  20  timeslice, faltam  0
E  rodou por  20  timeslice, faltam  0

--- Métricas por Processo ---
A -> Resposta: 0, Espera: 0, Execução: 20
B -> Resposta: 20, Espera: 20, Execução: 40
C -> Resposta: 40, Espera: 40, Execução: 60
D -> Resposta: 60, Espera: 60, Execução: 80
E -> Resposta: 80, Espera: 80, Execução: 100

Tempo médio de resposta: 40.00
Tempo médio de espera:   40.00
Tempo médio de execução: 60.00


({'A': 0, 'B': 20, 'C': 40, 'D': 60, 'E': 80},
 {'A': 0, 'B': 20, 'C': 40, 'D': 60, 'E': 80},
 {'A': 20, 'B': 40, 'C': 60, 'D': 80, 'E': 100})

---

Dado um workload de 5 processos de tamanho 20, calcule o tempo de execução do escalonador Round Robin com quantum=2:

Dado um workload de 5 processos de tamanho 20, calcule o tempo de resposta do escalonador Round Robin com quantum=2:



In [16]:
# Setup dos processos
nomes = ['A', 'B', 'C', 'D', 'E']
tamanho = [20] * 5
chanceio = [0] * 5
procs = [Processo(nomes[i], chanceio[i], tamanho[i], 0, 0) for i in range(5)]

# Simulação Round Robin
rr = EscalonadorRR(procs.copy(), quantum=2)
simular(rr, procs.copy(), usa_quantum=True)

A  rodou por  2  timeslice, faltam  18
B  rodou por  2  timeslice, faltam  18
C  rodou por  2  timeslice, faltam  18
D  rodou por  2  timeslice, faltam  18
E  rodou por  2  timeslice, faltam  18
A  rodou por  2  timeslice, faltam  16
B  rodou por  2  timeslice, faltam  16
C  rodou por  2  timeslice, faltam  16
D  rodou por  2  timeslice, faltam  16
E  rodou por  2  timeslice, faltam  16
A  rodou por  2  timeslice, faltam  14
B  rodou por  2  timeslice, faltam  14
C  rodou por  2  timeslice, faltam  14
D  rodou por  2  timeslice, faltam  14
E  rodou por  2  timeslice, faltam  14
A  rodou por  2  timeslice, faltam  12
B  rodou por  2  timeslice, faltam  12
C  rodou por  2  timeslice, faltam  12
D  rodou por  2  timeslice, faltam  12
E  rodou por  2  timeslice, faltam  12
A  rodou por  2  timeslice, faltam  10
B  rodou por  2  timeslice, faltam  10
C  rodou por  2  timeslice, faltam  10
D  rodou por  2  timeslice, faltam  10
E  rodou por  2  timeslice, faltam  10
A  rodou por  2  timeslic

({'A': 0, 'B': 2, 'C': 4, 'D': 6, 'E': 8},
 {'A': 72, 'B': 74, 'C': 76, 'D': 78, 'E': 80},
 {'A': 92, 'B': 94, 'C': 96, 'D': 98, 'E': 100})

---

# Questionário 02

Dado um workload com 4 processos de tamanhos 10, 20, 30 e 40, qual o tempo de execução do escalonador FIFO?

Dado um workload com 4 processos de tamanhos 10, 20, 30 e 40, qual o tempo de resposta do escalonador FIFO?



In [17]:
nomes = ['A', 'B', 'C', 'D']
tamanho = [10,20,30,40]
chanceio = [0] * 4
procs = [Processo(nomes[i], chanceio[i], tamanho[i], 0, 0) for i in range(4)]

# Simulação FIFO
fifo2 = EscalonadorFIFO(procs.copy())
simular(fifo2, procs.copy(), usa_quantum=False)

A  rodou por  10  timeslice, faltam  0
B  rodou por  20  timeslice, faltam  0
C  rodou por  30  timeslice, faltam  0
D  rodou por  40  timeslice, faltam  0

--- Métricas por Processo ---
A -> Resposta: 0, Espera: 0, Execução: 10
B -> Resposta: 10, Espera: 10, Execução: 30
C -> Resposta: 30, Espera: 30, Execução: 60
D -> Resposta: 60, Espera: 60, Execução: 100

Tempo médio de resposta: 25.00
Tempo médio de espera:   25.00
Tempo médio de execução: 50.00


({'A': 0, 'B': 10, 'C': 30, 'D': 60},
 {'A': 0, 'B': 10, 'C': 30, 'D': 60},
 {'A': 10, 'B': 30, 'C': 60, 'D': 100})

---

Dado um workload com 4 processos de tamanhos 10, 20, 30 e 40, qual o tempo de execução do escalonador SJF?

Dado um workload com 4 processos de tamanhos 10, 20, 30 e 40, qual o tempo de resposta do escalonador SJF?



In [48]:
nomes = ['A', 'B', 'C', 'D']
tamanho = [10,20,30,40]
chanceio = [0] * 4
procs = [Processo(nomes[i], chanceio[i], tamanho[i], 0, 0) for i in range(4)]

sjf2 = EscalonadorSJF(procs.copy())
simular(sjf2, procs.copy(), usa_quantum=False)

A  rodou por  10  timeslice, faltam  0
B  rodou por  20  timeslice, faltam  0
C  rodou por  30  timeslice, faltam  0
D  rodou por  40  timeslice, faltam  0

--- Métricas por Processo ---
A -> Resposta: 0, Espera: 0, Execução: 10
B -> Resposta: 10, Espera: 10, Execução: 30
C -> Resposta: 30, Espera: 30, Execução: 60
D -> Resposta: 60, Espera: 60, Execução: 100

Tempo médio de resposta: 25.00
Tempo médio de espera:   25.00
Tempo médio de execução: 50.00


({'A': 0, 'B': 10, 'C': 30, 'D': 60},
 {'A': 0, 'B': 10, 'C': 30, 'D': 60},
 {'A': 10, 'B': 30, 'C': 60, 'D': 100})

---

Dado um workload com 4 processos de tamanhos 10, 20, 30 e 40, qual o tempo de execução do escalonador Round Robin com quantum=2?

Dado um workload com 4 processos de tamanhos 10, 20, 30 e 40, qual o tempo de resposta do escalonador Round Robin com quantum=2?



In [49]:
nomes = ['A', 'B', 'C', 'D']
tamanho = [10,20,30,40]
chanceio = [0] * 4
procs = [Processo(nomes[i], chanceio[i], tamanho[i], 0, 0) for i in range(4)]

# Simulação Round Robin
rr2 = EscalonadorRR(procs.copy(), quantum=2)
simular(rr2, procs.copy(), usa_quantum=True)

A  rodou por  2  timeslice, faltam  8
B  rodou por  2  timeslice, faltam  18
C  rodou por  2  timeslice, faltam  28
D  rodou por  2  timeslice, faltam  38
A  rodou por  2  timeslice, faltam  6
B  rodou por  2  timeslice, faltam  16
C  rodou por  2  timeslice, faltam  26
D  rodou por  2  timeslice, faltam  36
A  rodou por  2  timeslice, faltam  4
B  rodou por  2  timeslice, faltam  14
C  rodou por  2  timeslice, faltam  24
D  rodou por  2  timeslice, faltam  34
A  rodou por  2  timeslice, faltam  2
B  rodou por  2  timeslice, faltam  12
C  rodou por  2  timeslice, faltam  22
D  rodou por  2  timeslice, faltam  32
A  rodou por  2  timeslice, faltam  0
B  rodou por  2  timeslice, faltam  10
C  rodou por  2  timeslice, faltam  20
D  rodou por  2  timeslice, faltam  30
B  rodou por  2  timeslice, faltam  8
C  rodou por  2  timeslice, faltam  18
D  rodou por  2  timeslice, faltam  28
B  rodou por  2  timeslice, faltam  6
C  rodou por  2  timeslice, faltam  16
D  rodou por  2  timeslice, falt

({'A': 0, 'B': 2, 'C': 4, 'D': 6},
 {'A': 24, 'B': 46, 'C': 58, 'D': 60},
 {'A': 34, 'B': 66, 'C': 88, 'D': 100})